# CMPUT 466/566, Winter 2020 Introduction to Machine learning 
## Coding Assignment 2 
### Problem 1 Report
By Nathan Klapstein #1449872

In [1]:

import struct

import numpy as np

import matplotlib.pyplot as plt
from scipy.special import expit


In [10]:

def readMNISTdata():
    with open('data/t10k-images-idx3-ubyte', 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        nrows, ncols = struct.unpack(">II", f.read(8))
        test_data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
        test_data = test_data.reshape((size, nrows * ncols))

    with open('data/t10k-labels-idx1-ubyte', 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        test_labels = np.fromfile(f,
                                  dtype=np.dtype(np.uint8).newbyteorder('>'))
        test_labels = test_labels.reshape((size, 1))

    with open('data/train-images-idx3-ubyte', 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        nrows, ncols = struct.unpack(">II", f.read(8))
        train_data = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
        train_data = train_data.reshape((size, nrows * ncols))

    with open('data/train-labels-idx1-ubyte', 'rb') as f:
        magic, size = struct.unpack(">II", f.read(8))
        train_labels = np.fromfile(f,
                                   dtype=np.dtype(np.uint8).newbyteorder('>'))
        train_labels = train_labels.reshape((size, 1))

    # augmenting a constant feature of 1 (absorbing the bias term)
    train_data = np.concatenate(
        (np.ones([train_data.shape[0], 1]), train_data), axis=1)
    test_data = np.concatenate((np.ones([test_data.shape[0], 1]), test_data),
                               axis=1)
    np.random.seed(314)
    np.random.shuffle(train_labels)
    np.random.seed(314)
    np.random.shuffle(train_data)

    X_train = train_data[:50000] / 256
    t_train = train_labels[:50000]

    X_val = train_data[50000:] / 256
    t_val = train_labels[50000:]

    return X_train, t_train, X_val, t_val, test_data, test_labels


In [12]:
 X_train, t_train, X_val, t_val, X_test, t_test = readMNISTdata()

In [32]:

def predict(X, W, t=None):
    # X_new: Nsample x (d+1)
    # W: (d+1) x K

    # TODO: Your code here
#     print(X, "w" W, t)
    
    # TODO: recycled code
#     y_hat = np.matmul(X, W)
#     loss = (1 / 2) * np.average(np.square(y_hat - y))
#     risk = np.average(np.abs(y_hat - y))

    t_hat = np.matmul(X, W)
    loss = (1 / 2) * np.average(np.square(t_hat - t))
    risk = np.average(np.abs(t_hat - t))
    # TODO: check if risk = acc
    acc = risk
    
    return y, t_hat, loss, acc


In [33]:

def train(X_train, y_train, X_val, t_val):
    N_train = X_train.shape[0]
    N_val = X_val.shape[0]

    # TODO: Your code here

    return epoch_best, acc_best, W_best


In [36]:

def gradient(X, w, y=None):
    # TODO: FIX:
    # TODO: FIX:  Did not scale gradient by 1/M;
    # TODO: FIX:
    return np.matmul(np.transpose(X), (np.matmul(X, w) - y))


In [ ]:

# TODO: recycled code
def train(X_train, y_train, X_val, y_val):
    N_train = X_train.shape[0]
    N_val = X_val.shape[0]

    # initialization
    w = np.zeros([X_train.shape[1], 1])
    # w: (d+1)x1

    losses_train = []
    risks_val = []

    w_best = None
    risk_best = 10000
    epoch_best = 0

    for epoch in range(MaxIter):
        loss_this_epoch = 0
        for b in range(int(np.ceil(N_train / batch_size))):
            X_batch = X_train[b * batch_size: (b + 1) * batch_size]
            y_batch = y_train[b * batch_size: (b + 1) * batch_size]
            # TODO: FIX:
            # TODO: FIX: Did not scale gradient by 1/M;
            # TODO: FIX:
            y_hat_batch, loss_batch, risk = predict(X_batch, w, y_batch)
            loss_this_epoch += loss_batch

            # TODO: Your code here
            # Mini-batch gradient descent
            w = w - alpha * gradient(X_batch, w, y_batch)
        #             print( "weight : " , w)
        #             print("loss_batch: " ,loss_batch)
        #             print("risk:" ,risk)

        # TODO: Your code here
        # monitor model behavior after each epoch
        
        # 1. Compute the training loss by averaging loss_this_epoch
        #         losses_train.append(loss_this_epoch/int(np.ceil(N_train/batch_size)))
        losses_train.append(loss_this_epoch / (int(np.ceil(N_train / batch_size))))
        #print(f"epoch={epoch} training loss={loss_this_epoch/  (int(np.ceil(N_train/batch_size)))}")

        # 2. Perform validation on the validation test by the risk
        risk_this_epoch = 0
        # TODO: FIX:
        # TODO: FIX: Validation set should not be run in batches
        # TODO: FIX:
        for c in range(int(np.ceil(N_val / batch_size))):
            X_val_batch = X_val[c * batch_size: (c + 1) * batch_size]
            y_val_batch = y_val[c * batch_size: (c + 1) * batch_size]
            _, _, risk = predict(X_val_batch, w, y_val_batch)
            risk_this_epoch += risk
        risks_val.append(risk_this_epoch / int(np.ceil(N_val / batch_size)))
        #print(f"epoch={epoch} risk={risk_this_epoch/int(np.ceil(N_val/batch_size))}")
        
        # 3. Keep track of the best validation epoch, risk, and the weights
        if risks_val[epoch] <= risk_best:
            epoch_best = epoch
            risk_best = risks_val[epoch]
            w_best = w

    # Return some variables as needed
    return epoch_best, risk_best, w_best

In [37]:
X_train, t_train, X_val, t_val, X_test, t_test = readMNISTdata()

print(X_train.shape, t_train.shape, X_val.shape, t_val.shape, X_test.shape,
  t_test.shape)

N_class = 10

alpha = 0.1  # learning rate
batch_size = 100  # batch size
MaxIter = 50  # Maximum iteration
decay = 0.  # weight decay

epoch_best, acc_best, W_best = train(X_train, t_train, X_val, t_val)

_, _, _, acc_test = predict(X_test, W_best, t_test)

print('At epoch', epoch_best, 'val: ', acc_best, 'test:', acc_test, 'train:', acc_train)


(50000, 785) (50000, 1) (10000, 785) (10000, 1) (10000, 785) (10000, 1)


NameError: name 'y' is not defined